# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Ваше ФИО`

Группа: `Ваша группа`

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)

2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [195]:
import folium
import random

import geopandas as gpd
import numpy as np 

from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from shapely.geometry import Point, LineString, Polygon

In [196]:
nominatim = Nominatim()
overpass = Overpass()

afghan_id = nominatim.query('Afghanistan').areaId()

query = overpassQueryBuilder(area=afghan_id, elementType=['relation'], selector='"name:UN:en"="Afghanistan"', includeGeometry=True)
result = overpass.query(query)

In [384]:
afghan = result.elements()[0]

afghan_border = np.array(afghan.geometry()['coordinates'][0])[:,::-1]
afghan_points = gpd.points_from_xy(afghan_border[:,0], afghan_border[:,1])
afghan_polygon = Polygon(afghan_border)

[min_lat_bbox, min_lng_bbox, max_lat_bbox, max_lng_bbox] = afghan_polygon.bounds
height_bbox = abs(abs(max_lat_bbox) - abs(min_lat_bbox))
width_bbox = abs(abs(max_lng_bbox) - abs(min_lng_bbox))

In [198]:
def gen_point(min_lat, min_lng, max_lat, max_lng):
    lat = random.uniform(min_lat, max_lat)
    lng = random.uniform(min_lng, max_lng)
    
    return lat, lng

In [613]:
map_afghan = folium.Map(location=[afghan_polygon.centroid.x, afghan_polygon.centroid.y], 
                        zoom_start=1.5, 
                        tiles = 'cartodbpositron')

afghan_border_group = folium.FeatureGroup(name='afghanistan border')
steps_of_search_point_group = folium.FeatureGroup(name='steps of search point')
point_within_afghanistan_group = folium.FeatureGroup(name='point within afghanistan')
search_bounds_group = folium.FeatureGroup(name='search bounds')
point_outside_afghanistan_group = folium.FeatureGroup(name='point outside afghanistan')

folium.Polygon(
    afghan_border,
    name='afghanistan border',
    color='red',
    opacity=0.5,
    fill=True,
    fill_color='red',
).add_to(afghan_border_group)

        
folium.CircleMarker(
    location=[(min_lat_bbox + max_lat_bbox) / 2, (min_lng_bbox + max_lng_bbox) / 2],
    name='afghanistan border',
    radius=2,
    popup=f'Latitude: {round(afghan_polygon.centroid.x, 5)}, Longitude: {round(afghan_polygon.centroid.y, 5)}',
    color='blue',
    fill=True,
    fill_color='blue',

).add_to(point_outside_afghanistan_group)

min_lat, min_lng, max_lat, max_lng = -90, -180, 90, 180

prev_point = []

while True:
    lat, lng = gen_point(min_lat, min_lng, max_lat, max_lng)
    
    if prev_point:
        folium.Polygon(
        [prev_point,[lat, lng]],
        name='steps of search point',
        color='brown',
    ).add_to(steps_of_search_point_group)
    
    prev_point = [lat, lng]
    
    if Point([lat, lng]).within(afghan_polygon):
        folium.CircleMarker(
            location=[lat, lng],
            name='point within afghanistan',
            radius=5,
            popup=f'Latitude: {round(lat, 5)}, Longitude: {round(lng, 5)}',
            color='green',
            fill=True,
            fill_color='green',
        ).add_to(point_within_afghanistan_group)
        break
    else:
        lat_plus = abs(abs(lat) - abs(afghan_polygon.centroid.x))
        lng_plus = abs(abs(lng) - abs(afghan_polygon.centroid.y))
        
        min_lat, min_lng = afghan_polygon.centroid.x - lat_plus, afghan_polygon.centroid.y - lng_plus
        max_lat, max_lng = afghan_polygon.centroid.x + lat_plus, afghan_polygon.centroid.y + lng_plus
               
    
    folium.Polygon(
        [[min_lat, min_lng], 
         [max_lat, min_lng],
         [max_lat, max_lng],
         [min_lat, max_lng]],
        name='search bounds',
        color='#67E667',
        opacity=0.01,
        fill=True,
        fill_color='#67E667',
        fill_opacity=0.2,
    ).add_to(search_bounds_group)
        
        
    folium.CircleMarker(
        location=[lat, lng],
        name='points outside afghanistan',
        radius=2,
        popup=f'Latitude: {round(lat, 5)}, Longitude: {round(lng, 5)}',
        color='red',
        fill=True,
        fill_color='red',
    
    ).add_to(point_outside_afghanistan_group)


search_bounds_group.add_to(map_afghan)
afghan_border_group.add_to(map_afghan)
point_within_afghanistan_group.add_to(map_afghan)
point_outside_afghanistan_group.add_to(map_afghan)
steps_of_search_point_group.add_to(map_afghan)

folium.LayerControl().add_to(map_afghan)

map_afghan

In [422]:
point_in_afghanistan = prev_point
point_in_afghanistan

[33.13547319984005, 68.50031260768685]

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [513]:
nominatim = Nominatim()
overpass = Overpass()

# earth_id = nomintaim.query()
# query = overpassQueryBuilder(area=earth_id.areaId(), elementType='relation', selector='"place"="country"')

result = overpass.query('relation["admin_level"="2"]["boundary"="administrative"]; out body;')

[overpass] downloading data: [timeout:25][out:json];relation["admin_level"="2"]["boundary"="administrative"]; out body;


In [529]:
id_countries = np.array(list(filter(lambda z: z % 10 == 0, map(lambda x: 3600000000 + x.id(),result.elements()))))

In [530]:
id_countries

array([3600059470, 3600079510, 3600080500, 3600088210, 3600090340,
       3600126410, 3600184640, 3600184840, 3600192780, 3600192790,
       3600192800, 3600192830, 3600195270, 3600195290, 3600196240,
       3600287670, 3600295480, 3600364110, 3600449220, 3600535790,
       3600535880, 3600536780, 3600536900, 3602088990, 3602103120,
       3602186600, 3602978650])

In [528]:
nominatim = Nominatim()
overpass = Overpass()

brasil_id = nominatim.query('Estonia').areaId()
brasil_id

[nominatim] downloading data: Estonia


3600079510

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [ ]:
# put your code here